In [62]:
from web3 import Web3, HTTPProvider
from eth_abi import encode
from eth_utils import keccak

In [63]:
RPC = "http://localhost:8545"

CONTRACT_ADDRESS = "0x5FbDB2315678afecb367f032d93F642f64180aa3"
ADDITION_SOLUTION_ADDRESS = "0x8464135c8F25Da09e49BC8782676a84730C318bC"

OWNER_PRIVATE_KEY = "0xac0974bec39a17e36ba4a6b4d238ff944bacb478cbed5efcae784d7bf4f2ff80"
OWNER_ADDRESS ="0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266"

SOLVER_ADDRESS = "0x70997970C51812dc3A010C7d01b50e0d17dc79C8"
SOLVER_PRIVATE_KEY = "0x59c6995e998f97a5a0044966f0945389dc9e86dae88c7a8412f4603b6b78690d"

w3 = Web3(HTTPProvider(RPC))

In [64]:
encoded_2 = encode(['uint256'], [2])
encoded_4 = encode(['uint256'], [4])
encoded_12 = encode(['uint256'], [12])

all_results = encoded_2 + encoded_4 + encoded_12

expected_results_hash = keccak(all_results).hex()

print(f"Expected results hash: {expected_results_hash}")

Expected results hash: 1f7d96dc808e221cdcfee683a0ce9dfc707f5a1ee10ec6febbcb3621cb1ac856


In [65]:
CONTRACT_ABI = [
    {"type":"function","name":"registerProblem","inputs":[{"name":"problemId","type":"uint256","internalType":"uint256"},{"name":"expectedResultsHash","type":"bytes32","internalType":"bytes32"},{"name":"testCases","type":"bytes[]","internalType":"bytes[]"},{"name":"title","type":"string","internalType":"string"},{"name":"description","type":"string","internalType":"string"}],"outputs":[],"stateMutability":"payable"},
    {"type":"function","name":"problemExists","inputs":[{"name":"problemId","type":"uint256","internalType":"uint256"}],"outputs":[{"name":"","type":"bool","internalType":"bool"}],"stateMutability":"view"},
    {"type":"function","name":"isProblemSolved","inputs":[{"name":"problemId","type":"uint256","internalType":"uint256"}],"outputs":[{"name":"","type":"bool","internalType":"bool"}],"stateMutability":"view"},
    {"type":"function","name":"getProblemPrize","inputs":[{"name":"problemId","type":"uint256","internalType":"uint256"}],"outputs":[{"name":"","type":"uint256","internalType":"uint256"}],"stateMutability":"view"}
]

w3 = Web3(HTTPProvider(RPC))
contract = w3.eth.contract(address=CONTRACT_ADDRESS, abi=CONTRACT_ABI)

In [66]:
def register_problem():
    problem_id = 1
    expected_results_hash = "0x1f7d96dc808e221cdcfee683a0ce9dfc707f5a1ee10ec6febbcb3621cb1ac856"
    title = "Addition Problem"
    description = "Solve the addition of two numbers. The expected results are the sum of the inputs provided in the test cases."

    test_case_1 = encode(['uint256', 'uint256'], [1, 1])
    test_case_2 = encode(['uint256', 'uint256'], [2, 2])
    test_case_3 = encode(['uint256', 'uint256'], [5, 7])

    test_cases = [
        test_case_1,
        # "0x00000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000001",
        test_case_2,
        test_case_3
    ]

    # Build transaction WITHOUT 'gas'
    tx = contract.functions.registerProblem(
        problem_id,
        expected_results_hash,
        test_cases,
        title,
        description
    ).build_transaction({
        'from': OWNER_ADDRESS,
        'value': w3.to_wei(0.1, 'ether'),
        'nonce': w3.eth.get_transaction_count(OWNER_ADDRESS),
    })

    try:
        gas_estimate = w3.eth.estimate_gas({
            'from': tx['from'],
            'to': tx['to'],
            'data': tx['data'],
            'value': tx['value'],
        })
        print(f"Estimated gas: {gas_estimate}")
        tx['gas'] = gas_estimate
    except Exception as e:
        message = str(e)
        if "revert reason:" in message:
            reason = message.split("revert reason:")[1].strip()
            print(f"Revert reason: {reason}")
        else:
            print(f"Transaction would revert. Error: {message}")

    signed_tx = w3.eth.account.sign_transaction(tx, private_key=OWNER_PRIVATE_KEY)
    tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
    print(f"Transaction sent: {tx_hash.hex()}")

    receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    if receipt.status == 1:
        print("Transaction was successful!")
    else:
        print("Transaction failed!")


# register_problem()

In [67]:
problem_id = 1
problem_exists = contract.functions.problemExists(problem_id).call()
if problem_exists:
    print(f"Problem {problem_id} exists.")

    prize = contract.functions.getProblemPrize(problem_id).call()
    print(f"Prize for problem {problem_id}: {Web3.from_wei(prize, 'ether')} ETH")
    
    is_solved = contract.functions.isProblemSolved(problem_id).call()
    print(f"Is problem {problem_id} solved? {'Yes' if is_solved else 'No'}")

Problem 1 exists.
Prize for problem 1: 100000000 ETH
Is problem 1 solved? No


In [75]:
!cast balance $CONTRACT_ADDRESS --rpc-url http://localhost:8545

100000000000000000000000000


In [68]:
!cast balance $ADDITION_SOLUTION_ADDRESS --rpc-url http://localhost:8545
!cast balance $CONTRACT_ADDRESS --rpc-url http://localhost:8545

0
100000000000000000000000000


In [69]:
!cast send $ADDITION_SOLUTION_ADDRESS --value 10000000000000000000 --from 0xa0Ee7A142d267C1f36714E4a8F75612F20a79720 --private-key 0x2a871d0798f97d79848a013d4936a73bf4cc922c825d33c1cf7073dff6d409c6 --rpc-url http://0.0.0.0:8545


blockHash            0x7342a46c7b2c0776ef35646ab8f5b9aeda3e08a27d70ccd46d6f3e888d346572
blockNumber          4
contractAddress      
cumulativeGasUsed    21055
effectiveGasPrice    693771496
from                 0xa0Ee7A142d267C1f36714E4a8F75612F20a79720
gasUsed              21055
logs                 []
logsBloom            0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
root                 
status               1 (success)
transactionHash      0xa06dbc115edc2e9cb1b6544ca0793f6af7170624b595e26baf1bc622644977b1
transactionInd

In [70]:
!cast balance $ADDITION_SOLUTION_ADDRESS --rpc-url http://localhost:8545

10000000000000000000


In [71]:
balance_before = w3.eth.get_balance(SOLVER_ADDRESS)
print(f"Owner balance: {Web3.from_wei(balance_before, 'ether')} ETH")

Owner balance: 9999999999999999998.999544115875458028 ETH


In [72]:
def submit_solution():
    balance = w3.eth.get_balance(ADDITION_SOLUTION_ADDRESS)
    print(f"Addition solution contract balance: {Web3.from_wei(balance, 'ether')} ETH")

    ABI = [
        {"type":"function","name":"submitSolution","inputs":[],"outputs":[],"stateMutability":"nonpayable"}
    ]
    solution_contract = w3.eth.contract(address=ADDITION_SOLUTION_ADDRESS, abi=ABI)
    tx = solution_contract.functions.submitSolution().build_transaction({
        'from': SOLVER_ADDRESS,
        'nonce': w3.eth.get_transaction_count(SOLVER_ADDRESS),
    })
    try:
        gas_estimate = w3.eth.estimate_gas({
            'from': tx['from'],
            'to': tx['to'],
            'data': tx['data'],
        })
        tx['gas'] = gas_estimate
    except Exception as e:
        print(f"Gas estimation failed: {e}")
    signed_tx = w3.eth.account.sign_transaction(tx, private_key=SOLVER_PRIVATE_KEY)
    tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
    print(f"Solution submitted! Transaction hash: {tx_hash.hex()}")
    receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    if receipt.status == 1:
        print("Solution submission was successful!")

        # call blaance of the addition solution contract
        balance_after = w3.eth.get_balance(ADDITION_SOLUTION_ADDRESS)
        print(f"Addition solution contract balance after submission: {Web3.from_wei(balance_after, 'ether')} ETH")
    else:
        print("Solution submission failed!")

def withdraw_rewards():
    ABI = [
        {"type":"function","name":"withdraw","inputs":[],"outputs":[],"stateMutability":"nonpayable"}
    ]
    solution_contract = w3.eth.contract(address=ADDITION_SOLUTION_ADDRESS, abi=ABI)
    tx = solution_contract.functions.withdraw().build_transaction({
        'from': SOLVER_ADDRESS,
        'nonce': w3.eth.get_transaction_count(SOLVER_ADDRESS),
    })
    try:
        gas_estimate = w3.eth.estimate_gas({
            'from': tx['from'],
            'to': tx['to'],
            'data': tx['data'],
        })
        tx['gas'] = gas_estimate
    except Exception as e:
        print(f"Gas estimation failed: {e}")
    signed_tx = w3.eth.account.sign_transaction(tx, private_key=SOLVER_PRIVATE_KEY)
    tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
    print(f"Withdraw called! Transaction hash: {tx_hash.hex()}")
    receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    if receipt.status == 1:
        print("Withdraw was successful!")
    else:
        print("Withdraw failed!")

submit_solution()
withdraw_rewards()

Addition solution contract balance: 10 ETH
Solution submitted! Transaction hash: 73cd87461611c91fffc38da027d222b243f010d9e56d663127c0f6d6574a98a6
Solution submission was successful!
Addition solution contract balance after submission: 100000010 ETH
Withdraw called! Transaction hash: 4a0fdf80fcd44ee0e53f65615aeb39298ba6ca1baa6e54141457b53ee599b737
Withdraw was successful!


In [73]:
balance_after = w3.eth.get_balance(SOLVER_ADDRESS)
print(f"Solver balance: {Web3.from_wei(balance_after, 'ether')} ETH")

if balance_after > balance_before:
    print("Solver's balance has increased.")

Solver balance: 10000000000100000008.999329545934390597 ETH
Solver's balance has increased.


In [74]:
problem_id = 1
problem_exists = contract.functions.problemExists(problem_id).call()
if problem_exists:
    print(f"Problem {problem_id} exists.")
    prize = contract.functions.getProblemPrize(problem_id).call()
    print(f"Prize for problem {problem_id}: {Web3.from_wei(prize, 'ether')} ETH")
    is_solved = contract.functions.isProblemSolved(problem_id).call()
    print(f"Is problem {problem_id} solved? {'Yes' if is_solved else 'No'}")

Problem 1 exists.
Prize for problem 1: 0 ETH
Is problem 1 solved? Yes
